In [1]:
import sys 
sys.path.append('..')
from cox.utils import Parameters
from cox.store import Store
from cox.readers import CollectionReader
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import itertools
import numpy as np
from scipy.spatial.distance import mahalanobis
import torch as ch
from torch import Tensor
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.utils.data import DataLoader
from delphi.stats.censored_multivariate_normal import censored_multivariate_normal
from delphi.utils.datasets import CensoredMultivariateNormalDataset as censored_dataset
from delphi.oracle import Lambda, horseshoe_dot
from delphi.utils.constants import COSINE
import datetime

# Helpers

In [19]:
STORE_PATH = '/home/pstefanou/horshoe_dot'
STORE_TABLE_NAME = 'censored_df'

store = Store(STORE_PATH)

store.add_table(STORE_TABLE_NAME, { 
    'delphi_kl': float,
    'naive_kl': float,
    'alpha': float, 
    'num_samples': int,
})

Logging in: /home/pstefanou/horshoe_dot/eba788a7-e7ad-469f-abe7-847d0236befc


## Default Experiment Parameters

In [20]:
# regression parameters
NUM_SAMPLES, DIMS = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 6000, 7000, 8000, 9000, 10000, 15000, 20000, 30000, 50000, 100000], 2
PREDICATE = Lambda(horseshoe_dot)

# procedure hyperparameters
args = Parameters({ 
    'epochs': 25,
    'num_workers': 0, 
    'batch_size': 10,
    'bias': True,
    'clamp': True, 
    'radius': 5.0, 
    'lr': 1e-1,
    'shuffle': False, 
    'tol': 1e-2,
    'custom_lr_multiplier': COSINE, 
    'phi': PREDICATE,
})

In [ ]:
# perform each experiment a total of 10 times
for iter_ in range(10): 
    for num_samples in NUM_SAMPLES:

        # sample from a 2 dimensional multivariate normal distribution until have num_samples in truncation set
        S = Tensor([])
        total_sampled = 0
        while S.size(0) < num_samples:
            M = MultivariateNormal(ch.zeros(DIMS), ch.eye(DIMS))
            samples = M.sample(ch.Size([num_samples]))
            S = ch.cat([S, samples[args.phi(samples).nonzero(as_tuple=False).flatten()]])[:num_samples]
            total_sampled += num_samples
        args.__setattr__('alpha', Tensor([S.size(0) / total_sampled]))
        
        # censored multivariate normal 
        data = censored_dataset(S)
        S = DataLoader(data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=args.shuffle)
        censored = censored_multivariate_normal(phi=args.phi, alpha=args.alpha, bias=args.bias, epochs=args.epochs, 
                                                      lr=args.lr, tol=args.tol, custom_lr_multiplier=COSINE)
        results = censored.fit(S)

        # metrics
        delphi_kl = ch.distributions.kl.kl_divergence(results, M)
        naive_kl = ch.distributions.kl.kl_divergence(MultivariateNormal(S.dataset.loc, S.dataset.covariance_matrix), M)

        store[STORE_TABLE_NAME].append_row({ 
            'delphi_kl': delphi_kl,
            'naive_kl': naive_kl,
            'alpha': float(args.alpha),
            'num_samples': num_samples,
        })

# close current store
store.close()

# Results

In [ ]:
reader = CollectionReader(STORE_PATH)
df = reader.df(STORE_TABLE_NAME)
reader.close() # close reader
df.head()

# All Trials

In [ ]:
print("all trials:")

# variance vs param mse
sns.lineplot(data=df, x='num_samples', y='delphi_kl', label='delphi', color='blue')
ax = sns.lineplot(data=df, x='num_samples', y='naive_kl', label='naive', color='red')
ax.set(xlabel='Number of Samples within Truncation Set', ylabel='KL Div with Mu*, Sigma*')
plt.show()

In [ ]:
results.loc, results.covariance_matrix

In [ ]:
plt.scatter(samples[:,0], samples[:,1], color="grey", alpha=.05, label="Ground-Truth")
emp = samples[PREDICATE(samples).nonzero(as_tuple=False).flatten()]
plt.scatter(emp[:,0], emp[:,1], color="red", label="S", alpha=.5)
pred = results.sample(ch.Size([10000]))
plt.scatter(pred[:,0], pred[:,1], label="predicted", color="green", alpha=.1)
# trunc = MultivariateNormal(S.dataset.loc, S.dataset.covariance_matrix).sample(ch.Size([1000]))
# plt.scatter(trunc[:,0], trunc[:,1], label="emp_dist", alpha=.25)
plt.legend() 

plt.show()

In [16]:
ch.distributions.kl.kl_divergence(results, M)

tensor(0.7173, grad_fn=<AddBackward0>)

In [18]:
ch.distributions.kl.kl_divergence(MultivariateNormal(S.dataset.loc, S.dataset.covariance_matrix), M)

tensor(0.9312)